In [ ]:
import json
import requests
import pandas as pd
import boto3
from datetime import datetime
from io import StringIO
# import io
import os
from bs4 import BeautifulSoup

# Create Functions to load this data
# Function for Name
def load_name(soup):
    Name = soup.find('h1',attrs ={'id':'pdp_product_title'}).text.strip()
    return Name

# Function for description
def load_description(soup):
    Description = soup.find('div', attrs = {'class':'description-preview body-2 css-1pbvugb'}).text.strip()
    return Description

# Function for Price
def load_price(soup):
    try:
        Price = soup.find('div',attrs ={'class':'product-price css-11s12ax is--current-price css-tpaepq'}).text.strip()
    except:
        Price = ''
    return Price
Nike = {'Name':[],'Price':[],'Description':[],'Link':[]}

def lambda_handler(event, context):
    Nike_URL = 'https://www.nike.com/w/mens-shoes-nik1zy7ok'
    HEADERS =({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
          'Accept-Language':'en-US, en; q =0.5'})
    # Request connection to wepage
    Nike_webpage = requests.get(Nike_URL, headers = HEADERS)
    Nike_webpage
    # Convert to HTML using beautifyul soup
    Nike_soup = BeautifulSoup(Nike_webpage.content, 'html.parser')
    # Get all links on the page
    links = Nike_soup.find_all('a', attrs={'class':'product-card__link-overlay'})
    # Loop through all the data to get just hyperlinks
    links_list = []
    for link in links:
        links_list.append(link.get('href'))
    for link in links_list:
#     Send Connection request to all links
        Nike_Webpages = requests.get(link, headers = HEADERS)
#     convert byte to html 
        Nike_Soup = BeautifulSoup(Nike_Webpages.content, 'html.parser')    
    
#     Get all releveant information using functions
        Nike['Name'].append(load_name(Nike_Soup))
        Nike['Description'].append(load_description(Nike_Soup))
        Nike['Price'].append(load_price(Nike_Soup))
        Nike['Link'].append(link)
    # print(Nike)
    Nike_df = pd.DataFrame.from_dict(Nike)
       
    bucket = 'nik-web-scrapping-project' # already created on S3
    csv_buffer = StringIO()
    Nike_df.to_csv(csv_buffer, index =False)
    s3_resource = boto3.client('s3')
    filename = "Nike_df" + str(datetime.now()) + ".csv"
    key =  "Nike_data/"  + filename
    # s3_resource.Object(bucket, 'Nike_df').put(Body=csv_buffer.getvalue())
    s3_resource.put_object(Bucket = bucket, Key = key, Body=csv_buffer.getvalue())
   

    
    